In [124]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
from flask import Flask, render_template, redirect, url_for
from flask_pymongo import PyMongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
from config import skey
import json
import pprint as pp
from collections import OrderedDict 


In [2]:
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [56]:
#shoutout to my friend for supplying our starting user id
initial_id = "76561198048037824"
friend_list_url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={skey}&steamid={initial_id}&relationship=friend"
first_query = requests.get(friend_list_url).json()

#initialize container for our steam ids
steamid_collection = []
for i in range(len(first_query["friendslist"]["friends"])):
    steamid_collection.append(first_query["friendslist"]["friends"][i].get("steamid"))
print(len(steamid_collection))


295


In [74]:
#now to find friends of friends
# for i in range(150):
    
#     #reset query url for next steam id in collection
#     friend_list_url = f"""http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={skey}&steamid={steamid_collection[i]}&relationship=friend"""
    
    
#     #new query from extracted steam id
#     try:
#         friend_query = requests.get(friend_list_url).json()
#         current_friendslist_length = len(friend_query["friendslist"]["friends"])
#         #print(current_friendslist_length)
#     except:
#         "KeyError"
        
#     #2nd loop for collecting current steam_id's friends
#     for j in range(current_friendslist_length):
#         try:
#             steamid_collection.append(friend_query["friendslist"]["friends"][j].get("steamid"))
#         except:
#             "KeyError"
            
            
#print(len(steamid_collection))
#print(len(set(steamid_collection)))
steamid_collection = set(steamid_collection)
steamid_collection = list(steamid_collection)
#print(steamid_collection[0])

17354
17354
76561198002210851


In [112]:
#dictionary to store game_titles and correspondingn appids
game_ids = {}

In [115]:
#function to identify each game from its appid
def demystify(appid):
    
    identifier = str(appid)

    #set base url for database that identifies games using appid
    base_appid_url = "https://steamdb.info/app/"
    
    #scrape the game title from steamdb.info
    browser.visit(base_appid_url + identifier)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    try:
        game_title = soup.find('h1').get_text()
    except:
        "KeyError"    
    game_ids.update({identifier : game_title})
    
    return game_title

In [116]:
print(demystify(221380))


Age of Empires II (2013)


In [117]:
print(game_ids)

{'221380': 'Age of Empires II (2013)'}


In [89]:
#function to collect account name and profile url
def basic_user_data(steamid):
    
    #construct container for user profile
    user_dictionary = OrderedDict({"user_steam_id" : steamid})
    
    #api url for player summary
    player_summary_url = (
        f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={skey}&steamids={steamid}"
                         )
    
    response = requests.get(player_summary_url).json()
    #extract data from json file
    try:
        user_dictionary.update({"current_account_name" : response['response']['players'][0].get("personaname")})
        user_dictionary.update({"original_account_name" : response['response']['players'][0].get("realname")})
        user_dictionary.update({"profile_url" : response['response']['players'][0].get("profileurl")})
    except:
        "KeyError"
        print(f"error at {steamid}")
    
    return user_dictionary

In [90]:
user_dictionary = basic_user_data(steamid_collection[0])
user_dictionary

OrderedDict([('user_steam_id', '76561198002210851'),
             ('current_account_name', 'NaturalHacks'),
             ('original_account_name', None),
             ('profile_url', 'https://steamcommunity.com/id/NaturalHacksNZ/')])

In [120]:
#function to collect specified users owned games and hours played for each game
def advanced_user_data(steamid, user_dictionary):
    
    #api url for user's game data
    owned_games_url = (
        f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={skey}&steamid=76561198048037824&format=json"
                      )
    game_list = requests.get(owned_games_url).json()
    
    #intialize temporary counter for game numbering
    play_count = 0   
    #intialize temporary dictionary container
    user_game_stats = {}
    
    for i in range(game_list["response"]["game_count"]):
    
        #initialize temporary variables
        appid = 0
        hours_played = 0
        try:
            if game_list["response"]["games"][i].get("playtime_forever") != 0:
                
                #pull game id
                appid = game_list["response"]["games"][i].get("appid")
                identifier = str(appid)
                #identify the game
                if identifier not in game_ids:
                    name_of_the_game = demystify(appid)
                elif identifier in game_ids:
                    name_of_the_game = game_ids[appid]
                    
                hours_played = game_list["response"]["games"][i].get("playtime_forever")
                #function to get appid actual name
                play_count += 1

                user_game_stats.update({name_of_the_game : {"appid" : appid,
                                                            "hours played" : "{:.2f}".format(hours_played/60)}
                                       })
        except:
            "KeyError"
                
            
    user_dictionary.update({"total_games_played" : play_count})
    user_dictionary.update({"user_game_data" : user_game_stats})
    return user_dictionary

In [103]:
finished_user_profile = advanced_user_data(steamid_collection[0], user_dictionary)
pp.pprint(finished_user_profile, indent = 4)

OrderedDict([   ('user_steam_id', '76561198002210851'),
                ('current_account_name', 'NaturalHacks'),
                ('original_account_name', None),
                (   'profile_url',
                    'https://steamcommunity.com/id/NaturalHacksNZ/'),
                ('total_games_played', 8),
                (   'user_game_data',
                    {   'game1': {'appid': 10180, 'hours played': '5.50'},
                        'game2': {'appid': 10190, 'hours played': '202.57'},
                        'game3': {'appid': 42680, 'hours played': '6.93'},
                        'game4': {'appid': 42690, 'hours played': '53.73'},
                        'game5': {'appid': 205790, 'hours played': '0.52'},
                        'game6': {'appid': 221380, 'hours played': '4.47'},
                        'game7': {'appid': 550, 'hours played': '0.37'},
                        'game8': {'appid': 544730, 'hours played': '0.03'}})])


In [105]:
print(finished_user_profile['user_game_data']['game1'])

{'appid': 10180, 'hours played': '5.50'}


In [79]:
#for the mongodb _id using .insert()
#str(steamid_collection[i])

In [121]:
user_dictionary = basic_user_data(steamid_collection[0])
finished_user_profile = advanced_user_data(steamid_collection[0], user_dictionary)
pp.pprint(finished_user_profile, indent=4)

OrderedDict([   ('user_steam_id', '76561198002210851'),
                ('current_account_name', 'NaturalHacks'),
                ('original_account_name', None),
                (   'profile_url',
                    'https://steamcommunity.com/id/NaturalHacksNZ/'),
                ('total_games_played', 7),
                (   'user_game_data',
                    {   'Call of Duty: Modern Warfare 2': {   'appid': 10180,
                                                              'hours played': '5.50'},
                        'Call of Duty: Modern Warfare 2 - Multiplayer': {   'appid': 10190,
                                                                            'hours played': '202.57'},
                        'Call of Duty: Modern Warfare 3': {   'appid': 42680,
                                                              'hours played': '6.93'},
                        'Call of Duty: Modern Warfare 3 - Multiplayer': {   'appid': 42690,
                                  